# Notebook to add leaks into train and save it

In [9]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
pd.options.display.max_colwidth = 50
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Creating base

In [10]:
new_test=pd.read_csv('..//bases/new_test_variants.csv')
new_test_texts = pd.read_csv('..//bases/new_test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
new_test_final=pd.merge(new_test,new_test_texts,how="left",on="ID")

In [14]:
leaks=pd.read_csv('..//bases/s1_add_train.csv')
leaks_1=pd.DataFrame([leaks["ID"],leaks.drop("ID",axis=1).idxmax(axis=1).map(lambda x: x.lstrip('class'))])
leaks_2=leaks_1.T
leaks_2.columns=["ID","Class"]

In [15]:
train = pd.read_csv('..//bases/training_variants')
test = pd.read_csv('..//bases/test_variants')

In [16]:
train_texts = pd.read_csv('..//bases/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
test_texts = pd.read_csv('..//bases/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")

In [17]:
train = pd.merge(train, train_texts, how='left', on='ID')
test = pd.merge(test, test_texts, how='left', on='ID')

In [18]:
leaks_3=pd.merge(leaks_2,test[test.ID.isin(leaks_2.ID)])
leaks_final=pd.merge(leaks_3,test_texts[test_texts.ID.isin(leaks_3.ID)])

In [19]:
new_train = pd.concat([train,leaks_final]) #adding first stage

In [25]:
new_train.index = range(len(new_train))
new_train.ID = new_train.index

new_train_variants = new_train[['Class', 'Gene', 'ID', 'Variation']]
new_train_text = new_train[['ID', 'Text']]
new_train_variants.to_csv("..//bases/new_training_variants.csv", index=False,encoding="utf8")
new_train_text.to_csv("..//bases/new_training_text.csv", index=False,encoding="utf8")

In [26]:
new_train

,Class,Gene,ID,Text,Variation
0,1,FAM58A,0,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations
1,2,CBL,1,Abstract Background Non-small cell lung canc...,W802*
2,2,CBL,2,Abstract Background Non-small cell lung canc...,Q249E
3,3,CBL,3,Recent evidence has demonstrated that acquired...,N454D
4,4,CBL,4,Oncogenic mutations in the monomeric Casitas B...,L399V
5,4,CBL,5,Oncogenic mutations in the monomeric Casitas B...,V391I
6,5,CBL,6,Oncogenic mutations in the monomeric Casitas B...,V430M
7,1,CBL,7,CBL is a negative regulator of activated recep...,Deletion
8,4,CBL,8,Abstract Juvenile myelomonocytic leukemia (JM...,Y371H
9,4,CBL,9,Abstract Juvenile myelomonocytic leukemia (JM...,C384R


In [ ]:
# All of the leaks are in the test set!
#len(set(leaks_final.Variation).intersection(set(new_test.Variation)) 
#len(leaks_final)

# Extra: finding the same examples of train in test

In [52]:
merge_match = new_test.merge(new_train, left_on=['Gene', 'Variation'], right_on = ['Gene', 'Variation'])
new_test.iloc[merge_match.ID_x - 1]

,ID,Gene,Variation
0,1,CHEK2,H371Y
1,2,AXIN2,Truncating Mutations
4,5,BRAF,T599insTT
9,10,AKT1,R328A
11,12,KIT,S709F
12,13,BRCA2,V211I
13,14,ALK,F1174C
20,21,PTPN11,N308D
23,24,AKT2,BCAM-AKT2 Fusion
25,26,MET,V1220I


In [47]:
submission = pd.read_csv('..//bases/stage2_sample_submission.csv')
class_probs = []

for i in range(1,10):
    class_probs.append(train[train['Class'] == i].shape[0]/3321.) ### Frequency of each class (total rows) divided by test set numbers
    
for i in range(9):
    submission['class'+str(i+1)] = class_probs[i]



In [56]:
Index_leak = merge_match.ID_x - 1
submission.iloc[Index_leak] = 1/9
submission.ID = range(len(submission))

In [57]:
submission

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,0,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
1,1,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
2,2,0.171033,0.136104,0.026799,0.206564,0.072870,0.082806,0.286962,0.005721,0.011141
3,3,0.171033,0.136104,0.026799,0.206564,0.072870,0.082806,0.286962,0.005721,0.011141
4,4,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
5,5,0.171033,0.136104,0.026799,0.206564,0.072870,0.082806,0.286962,0.005721,0.011141
6,6,0.171033,0.136104,0.026799,0.206564,0.072870,0.082806,0.286962,0.005721,0.011141
7,7,0.171033,0.136104,0.026799,0.206564,0.072870,0.082806,0.286962,0.005721,0.011141
8,8,0.171033,0.136104,0.026799,0.206564,0.072870,0.082806,0.286962,0.005721,0.011141
9,9,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
